# Data Acquisition and Preparation

## Table of Contents
1. [ZIP Code Tabulation Areas ](#zipcodetabulationareas)
2. [Commute Times](#commutetimes)
3. [Housing Costs](#housingcosts)
4. [Area Amenities](#areaamenities)
5. [Conclusions](#conclusions)

In this notebook we will explain how to acquire and prepare the data for our project. Some of our source data is only available as shapefiles [^1] which means that they include geographic coordinates for GIS software. This makes the files quite large, so we will strip out the geographic information along with records that we are not interested in (e.g. entries from other U.S. States), and generate a CSV file that contains the data that will be used for our analysis. 

Other source files contain a lot of extra columns that we are not interested in, so filtering them out before the analysis will reduce the amount of data loaded into memory later.

This will simplify the code that performs the analysis, and will reduce the processing and memory requirements.

[^1]: https://en.wikipedia.org/wiki/Shapefile

## ZIP Code Tabulation Areas <a name="zipcodetabulationareas"></a>

The official source for the U.S. ZIP Code Tabulation Areas (ZCTA) is the U.S. Census Bureau [^2], and the data files can be obtained from the **'TIGER/Line Shapefiles'** website [^3]. The latest version is the 2019 data file, that can be downloaded as a zip file. The next cell downloads the 2019 shapefile with the definition of the ZCTAs. Sometimes the U.S. Census website returns a *304 Not Modified* HTTP status instead of allowing for the download of the file (most likely this has to do with downloading the same file multiple times from the same location in a short period of time). To account for this contingency, we provide a copy of the file in the `source_data` directory.

[^2]: https://www.census.gov/
[^3]: https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html

In [11]:
# Uncomment the next line to download the ZCTA definition file from the US Census website
# !wget https://www2.census.gov/geo/tiger/TIGER2019/ZCTA5/tl_2019_us_zcta510.zip

We will process the shapefile using the GIS software of our choice. For this documentation, we will use QGIS [^4], a free open source GIS package available for Windows, Linux, BSD, and macOS.

![QGIS Logo](https://www.qgis.org/en/_downloads/qgis-logo.png "QGIS Logo")

[^4]: https://qgis.org

When we load the downloaded shapefile into QGIS, we see the graphical representation of the geographic objects in the file:

![ZCTA shapefile loaded in QGIS](./imgs/zcta_all_us_map.png "ZCTA shapefile loaded in QGIS")

If we inspect the features in the file, we get more information about the contents of the file we just loaded:

![Features of the ZCTA shapefile](./imgs/zcta_all_fields_table.png "Features in the ZCTA shapefile")

 

First, we need to acquire the State shapefile for the Census website. Similar as before, we provide a copy of the file in the `source_data` directory.

In [ ]:
# Uncomment the next line to download the States file from the US Census website
# !wget https://www2.census.gov/geo/tiger/TIGER2019/STATE/tl_2019_us_state.zip

After we load this new file into QGIS, we can see the geographic representation, similar to what we did for the ZCTAs:

![States shapefile loaded in QGIS](./imgs/states_all_us_map.png "States shapefile loaded in QGIS")

In the table of features we can find the information we were looking for:

![Features of the States shapefile](./imgs/states_all_fields_table.png "Features in the States shapefile")

For each entry, the states file contains the FIPS State Code (*STATEFP*, a two digit code that uniquely identifies each state and territory [^5]), the US Postal Service two-letter code for the states (*STUSPS*), and the fully spelled name (*NAME*). We can use any of these three fields in combination with the geography of the state to identify the ZIP codes that belong to each one of them. To do this, we will create a new layer (i.e. a new shapefile in memory) that will be the result of merging the fields of the state shapefile into the ZCTA shapefile, with the logic for the merge being that the ZCTA area is **WITHIN** the state area:

![Shapefile attribute merging parameters](./imgs/merging_parameters.png "Shapefile attribute merging parameters")


Once this process is over, we obtain a new layer that visually resembles the ZCAT shapefile:

![Visual aspect of the merged layer](./imgs/zcta_merged_us_map.png "Visual aspect of the merged layer")


But if we inpect the features for each shape we can see that the information from the state is now available for each entry:

![Features of the merged layer](./imgs/zcta_merged_fields_table.png "Features of the merged layer")


At this point, we can use QGIS filtering features to select and erase all the entries that are not part of the District of Columbia, Maryland, or Virginia, by looking at the value of the *STUSPS* field:

![Filtering the entries in the data file](./imgs/zcta_merged_filter.png "Filtering the entries in the data file")


After that, the resulting map of the entries looks much smaller than the original one:

![Map of the resulting entries](./imgs/zcta_filtered_map.png "Map of the resulting entries")


Now we can export the layer to a CSV file, and select the fields we want to export. In our case, we want the ZCTA identifier (*ZCTA5CE10*), the state identifier (*STUSPS*), and the coordinates of the internal point (*INTPTLAT10* and *INTPTLON10*):
![CSV export options](./imgs/zcta_export_options.png "CSV export options")


The resulting CSV file contains 1395 entries with the ZCTAs that belong to the District of Columbia, Maryland, or Virginia. A copy of this file (*zcta_md_dc_va.csv*) is provided in the `prepared_data` directory.

[^5]: https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code

In [12]:
! head prepared_data/zcta_md_dc_va.csv

ZCTA5CE10,INTPTLAT10,INTPTLON10,STUSPS
"23085","+37.6889537","-076.8256452",VA
"23086","+37.6685728","-077.0544136",VA
"23089","+37.4557955","-076.9035007",VA
"23091","+37.6562637","-076.8024918",VA
"23092","+37.6067689","-076.5147485",VA
"23093","+38.0041674","-078.0400574",VA
"23102","+37.7081193","-077.8286536",VA
"23103","+37.6480344","-077.7236401",VA
"23106","+37.7161066","-077.2015109",VA


## Commute Times <a name="commutetimes"></a>

In order to obtain the commute times for each ZCTA we will use the Bing Maps API [^6]. This API provides a free accounts with generous rate limits, so we can test the code and collect the data we need wihtout reaching the maximum number of queries. Using this API we only need to build an HTTP requests (GET or POST) providing our API key, the start and end coordinates of the commute, and the timestamp for the start or end of the commute. The query can contain multiple start points, and/or end points, and the server response will contain a matrix or individual responses that will account for those combinations. We will not use that syntax in our code, as the response times may increase significantly (although the number or billable transactions will decrease).

The server response is formatted as a JSON document, from which we want to extract the commute time. We will do this by using the `requests` package to build and send the GET request, and the JSON capabilities of the `pandas` package to parse the response and access the information we want. 

[^6]: https://docs.microsoft.com/en-us/bingmaps/rest-services/routes/calculate-a-distance-matrix

The commute information we are looking forward to obtain is the commute time from the internal point of each ZCTA to National Landing. From OpenStreetMaps [^7] we can see that the coordinates used for the map marker of National Landing are `(38.8548783, -77.0517428)` [^8]. This will be our commute end. The start will be the internal point that the ZCTA shapefile provided. Regarding the start time for the commute, we will evaluate the commute time at 30 minutes intervals, starting at 6:00am until 9:30am (included). We will also get the commute time over a work week (Monday through Friday), and then collapse the daily results into the average value for each time slot. That way we can capture the variation of commute times while still keeping the amount of features reasonable.

[^7]: https://www.openstreetmap.org
[^8]: https://www.openstreetmap.org/node/6061028362

The notebook [Preparation-CommuteTimes.ipynb](Preparation-CommuteTimes.ipynb) contains the code to load the list of ZCTAs from the CSV file generated in the previous section, obtain the average commute time at each start time, and save the results in another CSV file. We also provide the resulting CSV file in `prepared_data/zcta_commute_times.csv`.

> ⚠️ **Please note that running the `CommuteTimes.ipynb` takes several hours and consumes most of the allowed monthly queries for a month. Please, run with caution.** The resulting file is provided to ensure that there is no requirement to run that notebook

> ⚠️ The commute times were obtained during a period when the District of Columbia, Maryland, and Virginia had active stay-at-home orders, which could significantly affect the actual numbers obtained. Therefore, future executions of this notebook may yield different values than those in the CSV file provided.

Now that we have the commute times collected, we can try to trim our work data a bit more. If we look at the map of ZCTAs (with the coordinates of the National Landing marked as a red circle in the center), working with all the ZCTAs in Maryland and Virginia means that we are covering a huge area:

![Map of all the ZCTAs in the District of Columbia, Maryland, and Virginia](./imgs/zcta_commute_time_all.png "Map of all the ZCTAs in teh District of Columbia, Maryland, and Virginia")

Most of those locations are not really feasible locations to commute to the National Landing on a daily basis. We can appreciate this better by colorizing the map based on the maximum commute time of all the time slots:

![Map of all the ZCTAs in the District of Columbia, Maryland, and Virginia, colored by the maximum commute time](./imgs/zcta_commute_time_all_colorized.png "Map of all the ZCTAs in teh District of Columbia, Maryland, and Virginia, colored by the maximum commute time")

As we can see, there are huge regions with maximum commute times over 2 hours, not counting traffic delays, accidents, etc. So let's trim the selection of ZCTAs and let's work only with those that have a maximum commute time of 60 minutes. This reduces the number of areas to process from almost 1400 to only 181. 

![Map of the reduced selection of ZCTAs colored by the maximum commute time](./imgs/zcta_commute_time_reduced_colorized.png "Map of the reduced selection of ZCTAs colored by the maximum commute time")

The map we can look at now is more compact, allows us a finer granularity regarding the commute time clustering, and seems to line up better with natural features (we can tell the impact of crossing the river, or leaving the district), highways (the ZCTAs immediately next to the two main higways in the area, I-95 and I-270) show lower commute times than the neighbors), which is a better foundation for our analysis.

## Housing Costs <a name="housingcosts"></a>


[^9]: https://www.zillow.com/research/data/

The notebook [Preparation-HousingCosts.ipynb](Preparation-HousingCosts.ipynb) contains all the code for acquiring and preparing the housing costs data. In this notebook, the following datasets are downloaded from Zillow's web site:
  * ZHVI Time Series for Single Family Homes.
  * ZHVI Time Series for Condos/Co-op.
  * ZHVI Time Series for 1 Bedroom Houses.
  * ZHVI Time Series for 2 Bedroom Houses.
  * ZHVI Time Series for 3 Bedroom Houses.
  * ZHVI Time Series for 4 Bedroom Houses.
  * ZHVI Time Series for 5 or More Bedroom Houses.
  * ZRI Time Series for Multifamily, Single Family Residences, and Condos/Co-op.
Due to the large size of these files, a copy is provided in the `source_data` directory.

The notebook loads each of the CSV files into dataframes, and then uses the CSV with the commute times generated previously to filter the ZCTAs of interest to us. As it is possible that not all ZCTAs have housing information, we will filter the ones that lack this information from the commute times data set. As a result, the notebook generates 3 files:
  1. A new commute times CSV file with only the entries for those ZCTAs that have commute times and housing information.
  2. A housing information CSV file that contains all the index values for each of the ZCTAs under study.
  3. A CSV file with the commute and housing information, to be used as the master file for the analysis.


After the data processing, we see that several areas have been removed from the analysis. We can see how the maps of the buying and renting indexes look like:

Buying:

![Map of the ZCTAs colored by a house buying index](./imgs/zcta_housing_buy.png "Map of the ZCTAs colored by a house buying index")


Renting:

![Map of the ZCTAs colored by the house rent index](./imgs/zcta_housing_rent.png "Map of the ZCTAs colored by the house renting index")

## Area Amenities <a name="areaamenities"></a>

The information about the available amenities in each of the ZCTAs will be obtained using the FourSquare API. The notebook [Preparation-AreaAmenities.ipynb](Preparation-AreaAmenities.ipynb) containes the code used to acquire this information. As we have the coordinates for an internal point inside each of the ZCTAs, we will use those coordinates to query FourSquare for venues in a 500 m radius, with a maximum of 100 venues per area. We will then condense the results in a one-hot encoding like dataframe that will be subsequently aggregated per area, resulting in a dataframe listing all the types of venues for each area, along the percentage of total venues returned for the area that belong to that class.

Some of the areas queried returned no results, and therefore those entries have been removed from the list of areas for the analysis.

After all this, we saved two files, one with the housing indexes, commute times, and count of venues per area, and another one with the detailed breakdown of the venues in the area.

This is the map of the analysis areas, colorized using the total count of venues per area:

![Map of the ZCTAs colored by the venue count](./imgs/zcta_venues_count.png "Map of the ZCTAs colored by venue count")

## Conclusions <a name="conclusions"></a>

In this notebook we have detailed how to obtain the data we will use for the analysis of the Zip Code Tabulation Areas from the point of view of commute time, housing prices, and venues. By not only acquiring the data, but also cleaning up invalid or not available values, and trimming the original selection of ZCTAs depending on the values we have been obtaining for the different metrics, we have reduced the number of areas from 1395 to 102. This will allow our analysis to have a more adequate granularity, as well as be more meaningful.